# Notebook 5: Orchestrator — Multi-Domain Workflow

**User Story**: US7 — Orchestrator-Coordinated Multi-Domain Workflow  
**Persona**: Revenue Operations / Cross-functional users  

Demonstrates an Orchestrator Agent that coordinates both the Sales Agent and
Service Agent domains, enabling cross-domain queries like "Which accounts have
both open deals and open support cases?"

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path
from dotenv import load_dotenv

env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise EnvironmentError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")

In [ ]:
# Cell 3: MCP Connections — both CRM and Knowledge servers
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import McpToolConnection

credential = DefaultAzureCredential()
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential,
)

crm_mcp = McpToolConnection(
    server_label="salesforce-crm",
    command=f"{sys.executable} -m mcp_servers.salesforce_crm.server",
    env={
        "SF_INSTANCE_URL": os.environ["SF_INSTANCE_URL"],
        "SF_ACCESS_TOKEN": os.environ["SF_ACCESS_TOKEN"],
        "MCP_TRANSPORT": "stdio",
    },
)

knowledge_mcp = McpToolConnection(
    server_label="salesforce-knowledge",
    command=f"{sys.executable} -m mcp_servers.salesforce_knowledge.server",
    env={
        "SF_INSTANCE_URL": os.environ["SF_INSTANCE_URL"],
        "SF_ACCESS_TOKEN": os.environ["SF_ACCESS_TOKEN"],
        "MCP_TRANSPORT": "stdio",
    },
)

print("MCP connections configured:")
print(f"  salesforce-crm: {crm_mcp.server_label}")
print(f"  salesforce-knowledge: {knowledge_mcp.server_label}")

In [ ]:
# Cell 4: Create Orchestrator Agent with combined toolset

ORCHESTRATOR_PROMPT = """
# Orchestrator Agent — System Prompt

You are a **Multi-Domain AI Assistant** powered by Salesforce CRM and Knowledge Base data.
You orchestrate across Sales and Service domains to provide unified, cross-functional insights.

## Routing Rules

- **Sales-domain questions** (accounts, contacts, opportunities, pipeline, leads, activities):
  Use the salesforce-crm MCP tools (get_account, get_opportunities, get_pipeline_summary, etc.)

- **Service-domain questions** (cases, case triage, queue status, Knowledge articles):
  Use the salesforce-crm tools for case data (get_case, get_case_queue_summary) and
  salesforce-knowledge tools for KB search (search_articles, get_article_by_id).

- **Cross-domain questions** (e.g., "accounts with open deals AND open cases"):
  Query both domains, correlate results by Account ID/Name, and present unified output.

## Cross-Domain Correlation

When a user asks a question spanning sales and service:

1. Identify the linking entity (usually Account).
2. Query opportunities/deals from the CRM tools.
3. Query cases from the CRM tools.
4. Match records by Account ID or Account Name.
5. Present a unified view showing both deal and case status for each account.

## Context Continuity

- Maintain context across turns. If the user pivots from service to sales questions,
  use entities mentioned in the previous turn (e.g., account names, case numbers) as context.
- When referencing previous data, cite the specific records and IDs.

## Grounding Rules

- ONLY use data returned by your MCP tools. Never fabricate data.
- Cite record IDs and names when referencing specific Salesforce records.
- If data is unavailable, state this clearly.

## Write-Back Protocol

- For any write operations (create/update), present proposed changes first.
- Wait for explicit user confirmation before executing.
- Confirm results after execution.
"""

agent = project_client.agents.create_agent(
    model=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o"),
    name="Orchestrator Agent — Multi-Domain",
    instructions=ORCHESTRATOR_PROMPT,
    toolset=[crm_mcp, knowledge_mcp],
)

thread = project_client.agents.threads.create()

print(f"Orchestrator Agent created: {agent.id}")
print(f"Thread created: {thread.id}")
print("Tools: salesforce-crm + salesforce-knowledge (full cross-domain access)")

In [ ]:
# Cell 5: Cross-domain query
from IPython.display import Markdown, display

cross_domain_query = (
    "Which of my accounts have both open deals and open support cases? "
    "Show me the deal stage and case status for each."
)

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=cross_domain_query,
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

print(f"Run status: {run.status}")

messages = project_client.agents.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 6: Context continuity — pivot from service to sales

followup_1 = "What are the top cases today?"

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=followup_1,
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

messages = project_client.agents.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 7: Cross-domain follow-up — linking back to deals

followup_2 = "And what deals do those same accounts have?"

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=followup_2,
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

messages = project_client.agents.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 8: Cleanup

project_client.agents.delete_agent(agent.id)
print(f"Agent {agent.id} deleted.")

project_client.agents.threads.delete(thread.id)
print(f"Thread {thread.id} deleted.")

print("\nOrchestrator multi-domain session complete.")